### SQL Project

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers.

We've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

* [Connect to the database. Study the tables](#chapter1)
   
* [Find the number of books released after January 1, 2000.](#chapter2)

* [Find the number of user reviews and the average rating for each book.](#chapter3)

* [Identify the publisher that has released the greatest number of books with more than 50 pages](#chapter4)

* [Identify the author with the highest average book rating: look only at books with at least 50 ratings.](#chapter5)

* [Find the average number of text reviews among users who rated more than 50 books.](#chapter6)

* [Conclusions](#chapter7)

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
#function that prints the result
def result(query):
    return pd.io.sql.read_sql(query, con = engine)

* Books

In [3]:
books = '''SELECT * FROM books
           LIMIT 5'''
result(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


The books table contains data about book_id, author_id, title of the book, num_pages — number of pages in the book, publication_date and publisher_id

* Authors

In [4]:
authors = '''SELECT * FROM authors
           LIMIT 5'''
result(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


The authors table contains data on author_id, authors' name.

* Publishers

In [5]:
publishers = '''SELECT * FROM publishers
           LIMIT 5'''
result(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


The publishers table contains data on publisher_id, publishers' name.

* Ratings

In [6]:
ratings = '''SELECT * FROM ratings
           LIMIT 5'''
result(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


The raitings table contains data on rating_id, book_id, username — the name of the user who rated the book, rating.

* Reviews

In [7]:
reviews = '''SELECT * FROM reviews
           LIMIT 5'''
result(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


The reviews table contains data on review_id, book_id, username — the name of the user who reviewed the book, text — the text of the review

#### Find the number of books released after January 1, 2000.<a class="anchor" id="chapter2"></a>

In [8]:
query = ''' SELECT COUNT(book_id) as Number_of_books
            FROM books 
            WHERE publication_date > '2000-01-01' '''

result(query)

,number_of_books
0,819


In [9]:
query = ''' SELECT count(*) FROM books  '''

result(query)

,count
0,1000


 Out of 1000 books, 819 books were released after January 1, 2000 

#### Find the number of user reviews and the average rating for each book.<a class="anchor" id="chapter3"></a>

In [10]:
query = '''SELECT b.title as Title,
            COUNT(DISTINCT rv.review_id) as Number_of_reviews,
            AVG(rt.rating) as Average_rating 
            FROM books b 
            JOIN reviews rv ON b.book_id = rv.book_id  
            JOIN ratings rt ON b.book_id = rt.book_id
            GROUP BY b.book_id
            ORDER BY Number_of_reviews DESC'''

result(query)

,title,number_of_reviews,average_rating
0,Twilight (Twilight #1),7,3.662500
1,The Road,6,3.772727
2,The Book Thief,6,4.264151
3,The Glass Castle,6,4.206897
4,Water for Elephants,6,3.977273
...,...,...,...
989,Naked Empire (Sword of Truth #8),1,3.500000
990,Moo Baa La La La!,1,3.000000
991,Merrick (The Vampire Chronicles #7),1,4.000000
992,Babyville,1,3.500000


We can see that the most reviews get "Twilight", but it doesn't have the highest average rating. 

#### Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).<a class="anchor" id="chapter4"></a>

In [11]:
query = '''SELECT p.publisher as publisher,
COUNT(b.book_id) as number_of_books 
FROM books b 
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY number_of_books DESC
LIMIT 1'''

result(query)

,publisher,number_of_books
0,Penguin Books,42


"Penguin Books" publisher has released the greatest number of books (42 books) with more than 50 pages.

#### Identify the author with the highest average book rating: look only at books with at least 50 ratings.<a class="anchor" id="chapter5"></a>

In [12]:
query = '''SELECT a.author as author_name, b.title, AVG(r.rating) as avg_rating
            FROM authors a
            JOIN books b on b.author_id = a.author_id
            JOIN ratings r on r.book_id = b.book_id
            GROUP BY author_name, b.book_id
            HAVING COUNT(r.rating) >= 50
            ORDER BY avg_rating DESC
            LIMIT 1 '''

result(query)

,author_name,title,avg_rating
0,J.K. Rowling/Mary GrandPré,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634


J.K. Rowling/Mary GrandPré	with the highest average book rating (4,41) for "Harry Potter and the Prisoner of Azkaban"

In [13]:
query = '''SELECT a.author as author_name,
                  AVG(rt.average_rating) as avg_rate    
           FROM books b
           JOIN ( SELECT book_id, COUNT(rating) as cnt,
                         AVG(rating) as average_rating
                  FROM ratings
                  GROUP BY book_id ) as rt
           ON rt.book_id = b.book_id
           JOIN authors a ON a.author_id = b.author_id
           WHERE rt.cnt >= 50
           GROUP BY author_name
           ORDER BY avg_rate DESC
           LIMIT 1 '''

result(query)

,author_name,avg_rate
0,J.K. Rowling/Mary GrandPré,4.283844


#### Find the average number of text reviews among users who rated more than 50 books.<a class="anchor" id="chapter6"></a>

In [14]:
query = '''SELECT AVG(CNT_text) as avg_number_text_reviews
            FROM 
            (SELECT username
            FROM ratings 
            GROUP BY username
            HAVING COUNT(rating_id) > 50) as rt
            JOIN
            (SELECT username, COUNT(text) as CNT_text
            FROM reviews 
            GROUP BY username) rv
            ON rt.username = rv.username
            '''

result(query)

,avg_number_text_reviews
0,24.333333


The average number of text reviews among users who rated more than 50 books is 24 reviews.

#### Conclusions.<a class="anchor" id="chapter7"></a>

* Most of the books (819 out of 1000 books) were released after January 1, 2000

* "Twilight" got the most number of user reviews. At the top 5 of the number of user reviews we can see fantasy and historical novels. 

* "Penguin Books" publisher has released the greatest number of books (42 books) with more than 50 pages.

* J.K. Rowling/Mary GrandPré	with the highest average book rating (4,41) for "Harry Potter and the Prisoner of Azkaban".

* The average number of text reviews among users who rated more than 50 books is 24 reviews.